In [ ]:
import tensorflow as tf
import os
from tensorflow import keras
import keras_preprocessing as kp
import cv2
import imghdr

#https://www.kaggle.com/code/kausthubkannan/ai-human-art-classification-mobilenetv2-91/notebook

In [ ]:
tf.get_logger().setLevel('INFO')

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

In [ ]:

# Place tensors on the CPU
with tf.device('/CPU:0'):
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

# Run on the GPU
c = tf.matmul(a, b)
print(c)


In [ ]:
'''gpu_memory_fraction = 0.8  # Use 80% of the available GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth'''

In [ ]:
os.listdir('AIandRealArt/train/')
data_loc = 'AIandRealArt/train/'

In [ ]:
"""imgtypes = ['jpg','png','jpeg','bmp']
for iclass in os.listdir(data_loc):
    for image in os.listdir(os.path.join(data_loc,iclass)):
        image_path = os.path.join(data_loc,iclass,image)
        try:
            img = cv2.imread(image_path)
            typ = imghdr.what(image_path)
            if typ not in imgtypes:
                os.remove(image_path)
                print('Issue 1')
        except Exception as e:
            print("Issue")"""

In [ ]:
import numpy as np
from matplotlib import  pyplot as plt

In [ ]:
data = tf.keras.preprocessing.image_dataset_from_directory(data_loc,shuffle =True)

In [ ]:
data_it = data.as_numpy_iterator()

In [ ]:
batch = data_it.next()

In [ ]:
batch[0].shape

In [ ]:
fig, ax = plt.subplots(ncols = 4,figsize = (20,20))
for idx,img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

1 - Real
0 - AI

In [ ]:
data_scaled = data.map(lambda x,y :((x/255),y))

In [ ]:
scaled_it = data_scaled.as_numpy_iterator()

In [ ]:
batch_scaled = scaled_it.next()
batch_scaled[0]

In [ ]:
"""for batch in scaled_it:
    temp = list(batch)
    temp[0] = tf.keras.utils.normalize(temp[0], axis = 1)
    batch = tuple(batch)"""

In [ ]:
fig, ax = plt.subplots(ncols = 4,figsize = (20,20))
for idx,img in enumerate(batch_scaled[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch_scaled[1][idx])

In [ ]:
train_size = int(len(data_scaled)*0.7)
val_size = int(len(data_scaled)*0.1)+1
test_size = int(len(data_scaled)*0.2)+1
train_size+val_size+test_size

In [ ]:
train = data_scaled.take(train_size)
test = data_scaled.skip(train_size).take(test_size)
validation = data_scaled.skip(train_size).skip(test_size).take(val_size)

In [ ]:
train_it = train.as_numpy_iterator()
(train_it.next()[0])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Flatten,MaxPooling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import BatchNormalization

In [ ]:
data_aug = tf.keras.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        tf.keras.layers.experimental.preprocessing.RandomFlip("vertical"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.4),
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.5),
        tf.keras.layers.experimental.preprocessing.RandomContrast(0.2)
    ]
)
plt.figure(figsize=(10,10))
for images , _ in train.take(1):
    for i in range(9):
        augmented_images = data_aug(images)
        ax = plt.subplot(3,3,i+1)
        plt.imshow(augmented_images[0])

In [ ]:
model = Sequential()

In [ ]:
"""base_model = VGG16( include_top=False, input_shape=(224, 224, 3))"""

In [ ]:
"""model = Sequential()

# Block 1
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(256, 256, 3)))
model.add(MaxPooling2D((2, 2)))

# Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

# Block 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

# Block 4
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

# Block 5
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(GlobalAveragePooling2D())  # Global average pooling layer

model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))"""

In [ ]:
model.add(data_aug)

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))


model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())


model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
"""logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)"""

In [ ]:
hist = model.fit(train,epochs=20,validation_data=validation)

In [ ]:
model.summary()

In [ ]:
def pred_img(img):
    im = cv2.imread(img)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    resize = tf.image.resize(im,(256,256))
    yhat = model.predict(np.expand_dims(resize/255,0))
    if (yhat >= 0.5):
        plt.imshow(im)
        plt.title("Real")
    else:
        plt.imshow(im)
        plt.title("AI Generated")

In [ ]:
pred_img("test1.png") #AI

In [ ]:
pred_img("test2.jpeg") #Real

In [ ]:
pred_img("test3.jpg") #Real

In [ ]:
pred_img("Test4.png") # Real

In [ ]:
pred_img("test5.jpg") # Real

In [ ]:
pred_img("test6.png") #AI

In [ ]:
pred_img("test7.png") #AI

In [ ]:
pred_img("test8.png") #AI

In [ ]:
pred_img("test9.jpg") 

In [ ]:
pred_img("test10.jpg") 

In [ ]:
pred_img("test11.jpg") 

In [ ]:
pred_img("test12.jpg") 

In [ ]:
fig  = plt.figure()
plt.plot(hist.history["accuracy"],color='red')
plt.plot(hist.history["val_accuracy"],color='blue')
plt.show()

In [ ]:
fig  = plt.figure()
plt.plot(hist.history["loss"],color='red')
plt.plot(hist.history["val_loss"],color='blue')
plt.show()

In [ ]:
from tensorflow.keras.metrics import  Precision,BinaryAccuracy,Recall
pre = Precision()
ba = BinaryAccuracy()
Re = Recall()

In [ ]:
for i in test.as_numpy_iterator():
    X,y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    ba.update_state(y,yhat)
    Re.update_state(y,yhat)

In [ ]:
pre.result().numpy()

In [ ]:
ba.result().numpy()

In [ ]:
Re.result().numpy()